In [1]:
import numpy as np
import pandas as pd
import scipy.spatial
import scipy.cluster.hierarchy as sch
from IPython.core.display import SVG, HTML
import operator

In [2]:
import sys
from os.path import join as opj
from fg_shared import _git

sys.path.append(opj(_git, 'tcrdist2'))
from tcrdist.repertoire import TCRrep

sys.path.append(opj(_git, 'hierdiff'))
import hierdiff

sys.path.append(opj(_git, 'pwseqdist'))
import pwseqdist as pwsd

sys.path.append(opj(_git, 'seqlogo'))
import seqlogo

/home/agartlan/anaconda3/envs/py3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Using Cython-powered Fisher's exact test


In [3]:
df = pd.read_csv(opj(_git, 'hierdiff', 'dash.csv'))
df = tcrdist2_df[df.epitope.isin(['PB1', 'NP'])]

#2 create instance of TCRrep class, initializes input as tr.cell_df attribute
tr = TCRrep(cell_df=df, chains=['beta'], organism='human')

#3 Infer CDR1,CDR2,CDR2.5 (a.k.a. phmc) from germline v-genes
tr.infer_cdrs_from_v_gene(chain='beta',  imgt_aligned=True)

#4 Define index columns for determining unique clones.
tr.index_cols = ['clone_id', 'subject', 'epitope',
                 'v_b_gene', 'j_b_gene',
                'cdr3_b_aa', 'cdr1_b_aa', 'cdr2_b_aa', 'pmhc_b_aa',
                'cdr3_b_nucseq']

#4 Deduplicate based on index cols, creating tr.clone_df attribute
tr.deduplicate()
pw = tr.compute_pairwise_all('beta', metric='nw', processes=3)

NameError: name 'tcrdist2_df' is not defined

In [ ]:
res = hierdiff.neighborhood_diff(tr.clone_df,
                          pwmat=tr.cdr3_b_aa_pw,
                          x_cols=['epitope'],
                          count_col=None,
                          test='fishers',
                          knn_neighbors=100, knn_radius=None)
res = tr.clone_df.join(res)
res = res.sort_values(by='OR', ascending=False)
res[['epitope', 'v_b_gene', 'j_b_gene', 'cdr3_b_aa', 'x_val_1', 'x_freq_1', 'x_freq_0', 'OR', 'FWERp']]

In [ ]:
ind = 172
motif = seqlogo.compute_pal_motif(res.loc[ind, 'cdr3_b_aa'],
                        seqs=res.loc[res.loc[ind, 'neighbors'], 'cdr3_b_aa'],
                         refs=res.loc[[i for i in res.index if not i in res.loc[ind, 'neighbors']], 'cdr3_b_aa'])
html = seqlogo.svg_logo(motif, return_html=True)
HTML(data=html)

In [ ]:
ind = 203
motif = seqlogo.compute_pal_motif(res.loc[ind, 'cdr3_b_aa'],
                        seqs=res.loc[res.loc[ind, 'neighbors'], 'cdr3_b_aa'],
                         refs=res.loc[[i for i in res.index if not i in res.loc[ind, 'neighbors']], 'cdr3_b_aa'])
html = seqlogo.svg_logo(motif, return_html=True)
HTML(data=html)

In [ ]:
hdiff_res, Z = hierdiff.hcluster_diff(tr.clone_df,
                          pwmat=tr.cdr3_b_aa_pw,
                          x_cols=['epitope'],
                          count_col=None,
                          test='fishers',
                          method='ward')

summ_df = hierdiff.member_summ(res_df=hdiff_res, clone_df=tr.clone_df, summ_within=False)
hdiff_res = hdiff_res.join(summ_df)
hdiff_res = hdiff_res.sort_values(by='pvalue')